Let's do all preprocessing required for fitting the seq-2-seq model here. We need to extract features for the subwindows. We also need to write generic code incase we need to do changes later depending on the performance of the seq-2-seq model.

For the seq-2-seq model, we do not have to touch the densities, and as the first step we should extract the features from each subwindow using the fine-tuned model. We still might need the densities later.Therefore, we will also keep track of the densities, maybe for later use. We will need to arange the subwindows in a stack though, so that the features could be extracted. We ahve done this sort of work in AR models, we will look into that too.

In [2]:
import numpy as np
import tensorflow as tf
import os
from skimage.transform import resize

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

##### Train data

Let's first focus on train data. Meaning blocks 101, 102, 203, 301. We will preprocess data required for both stages, therefore, we will keep track of both subwindows and the densities.

In [ ]:
# Let's get block 0101 as the example, develop all functions for this, and wokr on preprocessing data